## Data ingestion pipeline

### LangChain dokument

2 glavne komponente: page_content(str) i metadata(dict)

#### Page content
sadrzaj dokumenta, mora bit string. Treba pazit na velicinu, ako je veliki dokument, onda je bolje raspodjelit na chunks

#### Metadata
Dodatne informacije o dokumentu, koriste se za filtriranje, pracenje i kontekst.
npr. izvor, timestamp, autor, jezik, chunk_id, itd.


In [3]:
from langchain_core.documents import Document


In [66]:
doc = Document(
    page_content="neki sadrzaj",
    metadata = {
        "source": "izvor.txt",
        "pages": 1,
        "author": "Pero Peric",
        "date_created": "2025-01-01"
    }
)

doc

Document(metadata={'source': 'izvor.txt', 'pages': 1, 'author': 'Pero Peric', 'date_created': '2025-01-01'}, page_content='neki sadrzaj')

### Ucitavanje tekst file-ova

In [24]:
## Text Loader
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./data/clanak.txt", encoding='utf-8')
document = loader.load()
print(document)

[Document(metadata={'source': './data/clanak.txt'}, page_content='Šefica Podravke Martina Dalić je na društvenim mrežama objavila da je danas prodano svih 350 dionica Podravke koje je pogreškom kupila u vrijeme zabrane trgovanja.\n\nNapisala je da su dionice prodane po prosječnoj cijeni od 155,97 eura, a ostvarena zarada iznosi 157,5 eura. Taj iznos je, prema njezinim riječima, doniran Autonomnoj ženskoj kući Zagreb. Uz ovaj iznos Dalić im je donirala i dodatnih 2000 eura.\nDodala je da je ova kupnja, kao i ona koja se dogodila osobnom pogreškom, transparentno objavljena na stranicama Zagrebačke burze, Hine i Podravke.\n\nPodsjetimo, nakon što je šefica Podravke Martina Dalić prije dva tjedna kupila dionice Podravke po cijeni od 155,32 eura, danas ih je odlučila prodati. Kupila je 350 dionica po cijeni od 155,52 eura, što je izazvalo niz reakcija. Dalić je dionice kupila neposredno uoči objave poslovnih rezultata, a onda se javno ispričala na LinkedInu.\n')]


In [33]:
# Directory loader

from langchain_community.document_loaders import DirectoryLoader
dir_loader = DirectoryLoader(
    "./data",
    glob="**/*.txt",
    loader_cls=TextLoader,
    loader_kwargs={'encoding': 'utf-8'}
)

documents=dir_loader.load()
documents

[Document(metadata={'source': 'data\\clanak.txt'}, page_content='Šefica Podravke Martina Dalić je na društvenim mrežama objavila da je danas prodano svih 350 dionica Podravke koje je pogreškom kupila u vrijeme zabrane trgovanja.\n\nNapisala je da su dionice prodane po prosječnoj cijeni od 155,97 eura, a ostvarena zarada iznosi 157,5 eura. Taj iznos je, prema njezinim riječima, doniran Autonomnoj ženskoj kući Zagreb. Uz ovaj iznos Dalić im je donirala i dodatnih 2000 eura.\nDodala je da je ova kupnja, kao i ona koja se dogodila osobnom pogreškom, transparentno objavljena na stranicama Zagrebačke burze, Hine i Podravke.\n\nPodsjetimo, nakon što je šefica Podravke Martina Dalić prije dva tjedna kupila dionice Podravke po cijeni od 155,32 eura, danas ih je odlučila prodati. Kupila je 350 dionica po cijeni od 155,52 eura, što je izazvalo niz reakcija. Dalić je dionice kupila neposredno uoči objave poslovnih rezultata, a onda se javno ispričala na LinkedInu.\n')]

### Ucitavanje svih pdf-a u direktoriju

In [34]:
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader # PyMuPDFLoader je bolja verzija od PyPDFLoader po brzini i nekim drugim metrikama

dir_loader = DirectoryLoader(
    "./data/pdf",
    loader_cls=PyMuPDFLoader,

)

pdf_documents = dir_loader.load()
pdf_documents

[Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-10-14T11:09:17+00:00', 'source': 'data\\pdf\\sap.pdf', 'file_path': 'data\\pdf\\sap.pdf', 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-14T11:09:17+00:00', 'trapped': '', 'modDate': 'D:20241014110917Z', 'creationDate': 'D:20241014110917Z', 'page': 0}, page_content='Statistiˇcka analiza podataka\nUpute za rad na projektu i izradu izvjeˇstaja\nRad na projektu\nPri radu na projektu vaˇzna je komponenta timski rad – iako ´ce se uvijek na´ci kolege koji ´ce se znanjem i\ntrudom istaknuti u svakoj grupi, svakako preporuˇcujemo da ukljuˇcite sve kolege u rad grupe, budu´ci da\nje znanje cijele grupe bitno pri konaˇcnoj predaji. U samom radu na dodijeljenoj temi vaˇzno je imati na\numu sljede´ce:\n• Pregledajte datoteke s podatcima i naˇcin na koji su sami podatci uˇcitani u vaˇsu razvojnu okolinu\n(RStudio) – uvijek pr

### Chunking

RecursiveCharacterTextSplitter ce nam omogucit da dokumente podjelimo u chunks.

On najprije pokusa razdvojit dokument po velikim separatorima poput \n, \n\n, itd. A ako to ne uspije onda predodredimo koliki chunk size da bude pa on odreze dokument i usred recenice ako je preslo npr. 1000 znakova

In [35]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# RecursiveCharacterTextSplitter ce nam omofucit da dokumente podjelimo u chunks


def split_documents(documents, chunk_size=1000,chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )

    split_docs = text_splitter.split_documents(documents)
    print(f"Splitao {len(documents)} dokumenta u {len(split_docs)} chunkova")

    if split_docs:
        print(f"Primjer chunk-a: ")
        print(f"Content: {split_docs[0].page_content[:200]}")
        print(f"Metadata: {split_docs[0].metadata}")

    
    return split_docs

In [36]:
chunks = split_documents(pdf_documents)

Splitao 15 dokumenta u 20 chunkova
Primjer chunk-a: 
Content: Statistiˇcka analiza podataka
Upute za rad na projektu i izradu izvjeˇstaja
Rad na projektu
Pri radu na projektu vaˇzna je komponenta timski rad – iako ´ce se uvijek na´ci kolege koji ´ce se znanjem i
Metadata: {'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-10-14T11:09:17+00:00', 'source': 'data\\pdf\\sap.pdf', 'file_path': 'data\\pdf\\sap.pdf', 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-14T11:09:17+00:00', 'trapped': '', 'modDate': 'D:20241014110917Z', 'creationDate': 'D:20241014110917Z', 'page': 0}


In [37]:
chunks

[Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-10-14T11:09:17+00:00', 'source': 'data\\pdf\\sap.pdf', 'file_path': 'data\\pdf\\sap.pdf', 'total_pages': 2, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-14T11:09:17+00:00', 'trapped': '', 'modDate': 'D:20241014110917Z', 'creationDate': 'D:20241014110917Z', 'page': 0}, page_content='Statistiˇcka analiza podataka\nUpute za rad na projektu i izradu izvjeˇstaja\nRad na projektu\nPri radu na projektu vaˇzna je komponenta timski rad – iako ´ce se uvijek na´ci kolege koji ´ce se znanjem i\ntrudom istaknuti u svakoj grupi, svakako preporuˇcujemo da ukljuˇcite sve kolege u rad grupe, budu´ci da\nje znanje cijele grupe bitno pri konaˇcnoj predaji. U samom radu na dodijeljenoj temi vaˇzno je imati na\numu sljede´ce:\n• Pregledajte datoteke s podatcima i naˇcin na koji su sami podatci uˇcitani u vaˇsu razvojnu okolinu\n(RStudio) – uvijek pr

### Embeddings

Kad smo tekst podjelili u chunkove, mozemo prijeci na embeddanje teksta u vektore.

Kao sto sam i u chroma testu napravio pomocu openAi embeddings modela, tako cu i tu.

In [38]:
import numpy as np
import chromadb
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity 
from dotenv import load_dotenv
import os
import openai
load_dotenv()

KEY=os.getenv('API_KEY')


In [39]:
import chromadb.utils.embedding_functions as embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
    api_key=KEY,
    model_name="text-embedding-3-small"
)

contents = [x.page_content for x in chunks] # Dobijemo clanke iz dokumenta
contents

embeddings = openai_ef(contents)

In [42]:
client = chromadb.PersistentClient(path="./vectordb")

collection = client.create_collection(name="predmeti")



In [43]:

def add_documents(documents, embeddings, collection):
    # funkcija za dodavanje dokumenata u kolekciju

    # documents je lista tipa Document

    if len(documents) != len(embeddings):
        raise ValueError("Broj dokumenata se mora podudarat sa brojem embeddinga")
    
    ids = []
    metadatas=[]
    documents_text=[]
    embeddings_list = []

    for i ,(doc, embedding) in enumerate(zip(documents, embeddings)):
        doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
        ids.append(doc_id)

        metadata=dict(doc.metadata)
        metadata['doc_index'] = i
        metadata['content_length'] = len(doc.page_content)
        metadatas.append(metadata)

        documents_text.append(doc.page_content)

        embeddings_list.append(embedding.tolist())
    
    try:
        collection.add(
            ids=ids,
            embeddings=embeddings_list,
            metadatas=metadatas,
            documents=documents_text
        )

    except Exception as e:
        print("error kod dodavanja dokumenta u collection")
        print(e)


    return collection

collection2 = add_documents(chunks, embeddings, collection)

print(collection2)


Collection(name=predmeti)


In [44]:
user_query = "Kad je kontrolna tocka"
query_embedded = openai_ef([user_query])
context = collection2.query(
    query_embeddings=query_embedded,
    n_results=3
)
print(context['documents'][0][0])

• Iako nema minimalne i maksimalne duljine izvjeˇstaja (a i ona uvelike ovisi o koliˇcini slika, tablica
i ispisa koje ´cete imati), ako vaˇs izvjeˇstaj ima previˇse ispisa koji su redundantni (npr. ispisi ili
upozorenja iz nekih funkcija u R-u koji nisu nuˇzno dio analize), moˇzete sprijeˇciti njihov ispis u
izvjeˇstaju, iako se funkcija izvrˇsava u kodu.
Kontrolna toˇcka i predaja projekta
U sklopu izrade projekta postoji kontrolna toˇcka koja je obavezna za sve grupe i za sve njihove
ˇclanove. U tom trenutku ´ce svaka grupa sa svojim asistentom pro´ci kroz svoj projekt (odnosno ono ˇsto
do tog trenutka imate), a asistenti ´ce dati preporuke za proˇsirenje ili popravak projekta. Za uspjeˇsnu
kontrolnu toˇcku svakako preporuˇcujemo da se pripremite s konkretnim pitanjima za asistente.
NAPOMENA: Kontrolna toˇcka se ne ocjenjuje i sasvim je dopuˇsteno do´ci s potpuno praznim projektom


S ovime smo napravili cjeli data ingestion pipeline.

Koristili smo LangChain za ucitavanje podataka iz pdf-a, teksta i ostlih sadrzaja. Taj tekst smo onda pretvorili u chunks i kasnije proveli embedding te vektore zapisali u vektorsku bazu pomocu chromadb

## Retrieval Pipeline

Sad mozemo krenut s drugim pipeline-om, a to je Retrieval, s kojim cemo handleat user_queries i doobavljanje iz vektorske baze

In [ ]:
class RAGRetriever:

    def __init__(self, collection):

        self.collection=collection
        print('kolekcija: ', collection)

    def retrieve(self, query, top_k=5, score_threshold=0.0):
        # Vrati relevantne dokumente na temelju query-a

        query_embedding = openai_ef([query])[0]

        try:
            results=collection.query(
                query_embeddings=[query_embedding],
                n_results=top_k
            )

            retrieved = []

            if results['documents'] and results['documents'][0]:
                # Stavljamo [0], zato sto se vraca u formatu [[]], pa da dobijemo pravu listu
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    similarity_score = 1/(1+distance) # ChromaDB koristi squared L2 distance da izracuna udaljenost
                    # 1/(1+distance) dobimo normaliziranu vrijednost izmedu 0 i 1

                
                    retrieved.append({
                        "id": doc_id,
                        "content": document,
                        "metadata": metadata,
                        "similarity_score": similarity_score,
                        'distance': distance,
                        'rank': i + 1
                    })

                print(f"Vratio {len(retrieved)} dokumenata nakon filtriranja")
            else:
                print("nema dokumenata")

            return retrieved
        
        except Exception as e:
            print(f"error kod dobavljanja dokumenata: {e}")
            return []


In [63]:
rag_retriever = RAGRetriever(collection2)

kolekcija:  Collection(name=predmeti)


In [64]:
test = rag_retriever.retrieve(query="Koji su predavaci za predmet PPJ")
test[0]
# Vidimo u content da se vratio ispravan chunk

Vratio 5 dokumenata nakon filtriranja


{'id': 'doc_fa6e2b3d_8',
 'content': '2 od 68\nCopyright © 2020 S.Srbljić et al.: Prevođenje programskih jezika\nNastavnici i opterećenje\nPredavači\nProf. dr. sc. Dejan Škvorc\nIzv. prof. dr. sc. Marin Šilić\nIzv. prof. dr. sc. Goran Delač\nIzv. prof. dr. sc. Klemo Vladimir\nDoc. dr. sc. Adrian Satja Kurdija\nAsistenti\nStjepan Požgaj, mag. inf. et math.\nECTS: 5\nOblik nastave\nOpterećenje\nPredavanja\n60\nLaboratorijske vježbe\n10',
 'metadata': {'keywords': '',
  'creator': '',
  'modDate': '',
  'title': '',
  'author': '',
  'producer': 'cairo 1.18.0 (https://cairographics.org)',
  'source': 'data\\pdf\\UNIZG-FER-PPJ-Uvodno.pdf',
  'page': 1,
  'content_length': 382,
  'file_path': 'data\\pdf\\UNIZG-FER-PPJ-Uvodno.pdf',
  'subject': '',
  'doc_index': 8,
  'total_pages': 13,
  'creationDate': "D:20251001133149+02'00",
  'creationdate': '2025-10-01T13:31:49+02:00',
  'trapped': '',
  'moddate': '',
  'format': 'PDF 1.7'},
 'similarity_score': 0.5339857825450229,
 'distance': 0.872

### LLM Output

Tu cemo spojit LLM te mu poslat kontekst iz vektorse baze

In [54]:
from langchain_openai import ChatOpenAI
# langchain ima integraciju sa openai pa mozemo samo api key staviti u njega
KEY=os.getenv('API_KEY')
llm = ChatOpenAI(model="gpt-4o-mini", api_key=KEY, temperature=0.1, model_kwargs={"max_tokens": 1024})
# Temperature oznacava koliko llm odstupa od najvjerojatnijeg tokena
# Ja sam stavio niski jer zelim smanjit halucinacije

# Napravimo RAG funkciju koja ce sve spojit
def rag(query, retriever, llm, top_k=3):

    result = retriever.retrieve(query, top_k=top_k)

    # Slozimo kontekst za LLM
    context = "\n\n".join([doc['content'] for doc in result]) if result else ""

    if not context:
        return "Nema relevantnog konteksta"
    
    # Generiramo odgovr sa LLM-om

    prompt = f"""
        You are a helpful assistant to students who want to find out information about their University, classes and other University related topics.
        Speak in Croatian.
        As a data source use only the context provided in this prompt.
        Speak in a helpful tone and offer your help.

        Context:
        {context}

        Question: {query}
         """
    
    response = llm.invoke([prompt.format(context=context, query=query)])

    return response.content


c:\Users\Korisnik\OneDrive\Radna površina\5.semestar\pror\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: Parameters {'max_tokens'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [55]:
answer = rag("Koja je minimalna duljina izvjestaja?", rag_retriever, llm)
print(answer)

Vratio 3 dokumenata nakon filtriranja
Iako nema minimalne duljine izvještaja, važno je da izvještaj bude jasan i sažet. Preporučuje se da se izbjegne previše redundantnih ispisa i da se fokusirate na bitne informacije. Ako imate dodatnih pitanja ili trebate pomoć oko izrade izvještaja, slobodno pitajte!


### Poboljsani RAG pipeline

In [56]:
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    result = retriever.retrieve(query, top_k=top_k, score_threshold = min_score)

    if not result:
        return {'answer': "Nema relevantnog konteksta", 'sources': [], 'confidence': 0.0, 'context': ''}
    
    context = "\n\n".join([doc['content'] for doc in result])

    sources = [{
        'source': doc['metadata'].get('source', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get("page", "unknown"),
        'score': doc['similarity_score'],
        'preview': doc['content'][:120] + '...'
        } for doc in result]
    
    confidence = max([doc['similarity_score'] for doc in result])

    prompt = f"""
        You are a helpful assistant to students who want to find out information about their University, classes and other University related topics.
        Speak in Croatian.
        As a data source use only the context provided in this prompt.
        Speak in a helpful tone and offer your help.

        Context:
        {context}

        Question: {query}
         """
    
    response = llm.invoke([prompt.format(context=context, query=query)])

    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }

    if return_context:
        output['context'] = context

    return output




In [65]:
result = rag_advanced("Sto se radi na predmetu PPJ?", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)


print("Answer: ", result['answer'])
print("Sources: ", result['sources'])
print("Confidence: ", result['confidence'])
print("Context preview: ", result['context'][:300])



Vratio 3 dokumenata nakon filtriranja
Answer:  Na predmetu "Prevođenje programskih jezika" (PPJ) studenti se bave raznim aspektima razvoja jezičnih procesora. Tijekom nastave, fokusira se na semantičku analizu, generiranje međukoda i ciljnog programa, te optimizaciju. Predavanja su organizirana u blokovima, a studenti sudjeluju u laboratorijskim vježbama gdje rade na zahtjevnim projektnim zadacima u grupama. 

Također, postoji kontinuirana provjera znanja koja uključuje rad u grupi, projektne zadatke, te usmena ispitivanja kako bi se provjerio pojedinačni doprinos članova grupe. Ako imate dodatnih pitanja ili trebate više informacija o predmetu, slobodno pitajte!
Sources:  [{'source': 'data\\pdf\\UNIZG-FER-PPJ-Uvodno.pdf', 'page': 5, 'score': 0.5266247988025364, 'preview': '6 od 68\nCopyright © 2020 S.Srbljić et al.: Prevođenje programskih jezika\nRed predavanja\nDRUGI CIKLUS NASTAVE\n8. tjedan\nP...'}, {'source': 'data\\pdf\\UNIZG-FER-PPJ-Uvodno.pdf', 'page': 8, 'score': 0.52288542520

## Zakljucak

S ovime sam prosao cjeli RAG pipeline uz pomoc LangChain i ChromaDB.

1. Data ingestion pipeline
Ucitavanje podataka uz PyMuPDFLoader i DirectoryLoader.
Chunking sa RecursiveCharacterTextSplitter za djeljenje dokumenata u manje dijelove.
Embeddanje sa OpenAI modelom.
ChromaDB PersistentClient za pohranu vektora lokalno u bazu.
ChromaDB collections za pohranu dokumenata, njihovih embeddinga i meta podataka. Takoder se koristi i za queryanje.

2. Retrieval pipeline
RagRetriever klasa koji uzme user query, napravi embedding i dohvati relevantne podatke. Vraca njihov metadata i content. Takoder racuna i similarity score.

3. Generation
Na kraju imamo i rag_advanced funkciju koja prima RagRetriever klasu, user query i ostale metrike.
Dohvaca relevantne podatke te salje prompt LLM-u sa dohvacenim kontekstom.
LLM generira odgovor i on se prikazuje korisniku
